In [1]:
import gnp
import codecs
import json
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import boto
import sys, os
import pandas as pd
from TwitterSearch import *
from sunlight import congress
from bs4 import BeautifulSoup
import urllib2
import itertools
import datetime
import configparser
import time
from pattern.en import parsetree
from pattern.en import mood
import dateutil.parser as dparser
import sunlight
from sunlight import congress
import re
from postal.parser import parse_address

In [14]:
config = configparser.ConfigParser()
config.read(os.path.join(os.pardir, 'config', 'ross.ini'))

consumer_key = config['twitter.api']['consumer_key']
consumer_secret = config['twitter.api']['consumer_secret']
access_token = config['twitter.api']['access_token']
access_token_secret = config['twitter.api']['access_token_secret']

AWS_ACCESS_KEY_ID = config['aws.creds']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['aws.creds']['AWS_SECRET_ACCESS_KEY']

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  from ipykernel import kernelapp as app


In [3]:
s = parsetree('You should call your senator today.')
for sentence in s:
  for chunk in sentence.chunks:
    print chunk.type, [(w.string, w.type) for w in chunk.words]

NP [(u'You', u'PRP')]
VP [(u'should', u'MD'), (u'call', u'VB')]
NP [(u'your', u'PRP$'), (u'senator', u'NN'), (u'today', u'NN')]


In [4]:
mood('Call your senator today!')

'imperative'

In [6]:
# issues = ["civil rights",
# "women's rights","planned parenthood",
# "black lives matter","african american rights","black rights",
# "immigration","immigrants","undocumented","border wall","refugees","refugee rights",
# "gun control",
# "public education",
# "LGBTQ rights","LGBTQ","LGBT",
# "voting rights","redistricting","voter id laws","voting access","voter access"
#          ]


action_categories = {'Donation': ["donation","give","donate","give support","financial support","contribute","contributing"],
                     'Protest': ["protest","rally","demonstration","demonstrate"],
                     'Gathering': ["meetup","huddle","congregate","gather","gathering","discuss","discussion"],
                     'Boycott': ["boycott"],
                     'Advocate': ["call","email","reach out","senator","representative", "sign petition","petition"],
                     'Vote': ["vote","cast your ballot"],
                     'Townhall': ["town hall","open office","town meeting","townhall",'virtual townhall','listening session']
                          }

issue_types = { 'immigrant rights' : ['immigrant rights',"immigrants' rigths",'refugee rights', 'travel ban',
                                        'border wall','refugees','asylum','immigration reform',
                                        'immigrant advocacy','migrant rights'],
               "women's rights" : ["women's rights","women's rights",'womens rights','gender equality'
                                  'girl power',"international women's day","war on women","planned parenthood"],
               "black rights" : ["racial equality","black lives matter","african american rights","civil rights",
                                "black power","jim crow"],
               "LGTBQ rights" : ["marriage equality","transgender rights","equality act","lesbian rights",'gay rights',
                                'bisexual rights'],
               "voting rights": ["redistricting","gerrymandering","redistrict","gerrymander","voter id","voting access",
                                "voter access","voter suppression"]
                      }


In [7]:
action_categories.keys()
listed = action_categories['Townhall']
listed.append('Ross')
print listed


['town hall', 'open office', 'town meeting', 'townhall', 'virtual townhall', 'listening session', 'Ross']


In [8]:
legislators = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'legislators.csv'))

mask = (legislators.in_office == 1)

legislators = legislators.loc[mask]

legislators['title_name'] = legislators['full_title'] + ' ' + legislators['firstname'] + ' ' + legislators['lastname']

print legislators.columns
print
print list(legislators['title_name'])[:10]

Index([u'title', u'full_title', u'firstname', u'middlename', u'lastname',
       u'name_suffix', u'nickname', u'party', u'state', u'district',
       u'in_office', u'gender', u'phone', u'fax', u'website', u'webform',
       u'congress_office', u'bioguide_id', u'votesmart_id', u'fec_id',
       u'govtrack_id', u'crp_id', u'twitter_id', u'congresspedia_url',
       u'youtube_url', u'facebook_id', u'official_rss', u'senate_class',
       u'birthdate', u'oc_email', u'title_name'],
      dtype='object')

['Representative Robert Aderholt', 'Senator Lamar Alexander', 'Representative Justin Amash', 'Representative Mark Amodei', 'Representative Alma Adams', 'Representative Pete Aguilar', 'Representative Rick Allen', 'Representative Ralph Abraham', 'Representative Jodey Arrington', 'Representative Joe Barton']


In [9]:
immigration_orgs = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'immigration_orgs.csv'))

# for org in immigration_orgs['org_name']:
#   print org


In [10]:
leg_names = list(legislators['title_name'])
leg_states = list(legislators['state'])
leg_twitter = list(legislators['twitter_id'])

keyword_searches = []

for combination in itertools.product(action_categories, issues):
  keyword_searches.append(list(combination))
  
print keyword_searches[:20]

NameError: name 'issues' is not defined

In [17]:
today = datetime.datetime.today()
weeks_ago = today - datetime.timedelta(days=14)
errors = 0
sleep_for = 3*60 # sleep for 3 min
last_amount_of_queries = 0 # used to detect when new queries are done

  
with open(os.path.join(os.pardir, 'data/documents', 'real_data_' + today.strftime("%Y%m%d%H%M") + '.txt'), 'w') as outfile:
  outfile.write('[')
  ntweets = 0

  for org in immigration_orgs['org_name']:
    for category in action_categories.keys():
      listed = action_categories[category]
      try:
        tso = TwitterSearchOrder() # create a TwitterSearchOrder object
  #         tso.set_keywords(action_categories[category], or_operator = True) # let's define all words we would like to have a look for
        tso.set_keywords(listed, or_operator = True)
        tso.add_keyword('AND '+org)
        tso.add_keyword('AND -filter:retweets')
        tso.set_language('en') # we want to see German tweets only
        tso.set_include_entities(True) # and don't give us all those entity information
        tso.set_result_type('recent')
  #       set_geocode(52.5233,13.4127,10,imperial_metric=True)
        tso.set_count(10)
  #       tso.set_link_filter()
        querystr = tso.create_search_url()


        tso2 = TwitterSearchOrder()
        tso2.set_search_url(querystr + '+since:'+ weeks_ago.strftime("%Y-%m-%d"))
  #        -filter:retweets
          # it's about time to create a TwitterSearch object with our secret tokens
        ts = TwitterSearch(
            consumer_key,
            consumer_secret,
            access_token,
            access_token_secret
         )

        ### this is where the fun actually starts :)
        for i,tweet in enumerate(ts.search_tweets_iterable(tso2)):
          
#           ### Extract URL from Tweet if it exists
#           try:
#             if tweet.get('entities',False).get('media',False):
#               tweet_url = tweet['entities']['media'][0]['url']
#             else:
#               tweet_url = ''
#           except:
#             tweet_url = ''

        ### Extract Date from Tweet if it exists
          try:
            tweet_date = dparser.parse(tweet['text'],fuzzy=True).strftime("%m/%d/%Y")
          except:
            tweet_date = None
        ### Extract Address from Tweet if it exists
          try:
            address_components = {}
            for each in parse_address(tweet['text']):
              address_components[each[1]] = each[0]
            address = address_components.get('house_number','') + ' ' + address_components.get('road','')
            city = address_components.get('city',None)
            state = address_components.get('state',None)
            zipcode = address_components.get('postcode',None)
          except:
            address = ''
            city = None
            state = None
            zipcode = None
        ### Extract City and State from User Location
          try:
            if city == None:
              matches = re.findall("([\w\s]+),\s(\w+)", string)
              match = matches[0]
              tweet_city = match[0]
            if state == None:
              matches = re.findall("([\w\s]+),\s(\w+)", string)
              match = matches[0]
              tweet_state = match[1]
          except:
            tweet_city = ''
            tweet_state = ''
          ### Get Hashtags
          try:
            if tweet.get('entities',False).get('hashtags',False):
              tweet_hashtags = tweet['entities']['hashtags']
            else:
              tweet_hashtags = []
          except:
            tweet_hashtags = []

          json_doc = {
                            'posted_date': datetime.datetime.today().strftime("%m/%d/%Y"),
                            'expiration_date': tweet_date,
                            'action_keywords': action_categories[category], 
                            'issues_keywords': issue_types['immigrant rights'], 
                            'document': {'content': tweet['text'],
                                       'title': None,
                                       'summary': None,
                                       'url': 'https://twitter.com/statuses/' + str(tweet['id']),
                                       'date': tweet['created_at']
                                    },
                            'location': {'addr1': address,
                                       'addr2': None,
                                       'city':  tweet_city,
                                       'state': tweet_state,
                                       'zip':  zipcode ,
                                       'lat':   None,
                                       'long':  None,
                                       },
                            'org_name': org,

                            # your mileage may vary
                            'other_metadata': {
                                'twitter_coordinates': tweet['coordinates'],
                                'twitter_hashtags': tweet_hashtags,
                                'twitter_favorite_count': tweet['favorite_count'],
                                'twitter_retweet_count': tweet['retweet_count']
                                }
                        }

          new_record = {
                                "_index" : "twitter",
                                "_type"  : "tweet",
                                "_id"    : tweet['id'],
                                "_source": json_doc,
                            }

          json.dump(new_record, outfile)
          outfile.write(',')
          ntweets += 1
          if ntweets % 100:
            print( tweet['text']  )
          
          ### Manage Rate Limits
          current_amount_of_queries = ts.get_statistics()[0]
          if not last_amount_of_queries == current_amount_of_queries:
            last_amount_of_queries = current_amount_of_queries
            time.sleep(sleep_for)

      except TwitterSearchException as e: # take care of all those ugly errors if there are some
        if errors == 5:
          break
        print(e)
        errors + 1
        time.sleep(60*15)
  outfile.seek(-1, os.SEEK_END)
  outfile.truncate()
  outfile.write(']')

Error 429: ('Too Many Requests: Request cannot be served ', "due to the application's rate limit having ", 'been exhausted for the resource')
Error 429: ('Too Many Requests: Request cannot be served ', "due to the application's rate limit having ", 'been exhausted for the resource')
RT @IOMAustralia: TOGETHER! IOM, UNIC and UNHCR at the Canberra Multicultural
Festival. Many join each year to celebrate culture!#UN4Refuge…
RT @IOMAustralia: TOGETHER! IOM, UNIC and UNHCR at the Canberra Multicultural
Festival. Many join each year to celebrate culture!#UN4Refuge…
RT @UNHCR_SriLanka: What do you want to ask internationally acclaimed writer and UNHCR supporter Neil Gaiman? Join us, and host... https://…
RT @Aussie4Refugees: Refugee Council tweet this horror, and UNHCR claim "profound concern", so why not call for an END TO THE POLICY?  http…
Error 429: ('Too Many Requests: Request cannot be served ', "due to the application's rate limit having ", 'been exhausted for the resource')
Error 429:

KeyboardInterrupt: 

In [23]:
#,'ACLU', 'CAPAction'

# try:
#     tuo = TwitterUserOrder('ACLU') # create a TwitterUserOrder
    
#     # it's about time to create TwitterSearch object again
#     ts = TwitterSearch(
#           consumer_key,
#           consumer_secret,
#           access_token,
#           access_token_secret
#     )

#     # start asking Twitter about the timeline
#     for i,tweet in enumerate(ts.search_tweets_iterable(tuo)):
#       print( '@%s tweeted: %s' % ( tweet['user']['screen_name'], tweet['text'] ) )       
#       if i > 20:
#         break

# except TwitterSearchException as e: # catch all those ugly errors
#     print(e)
    

In [60]:
# conn = S3Connection(AWS_ACCESS_KEY_ID,AWS_SECRET_ACCESS_KEY)
# bucket = conn.get_bucket('mids-w205-assignment-3-new-tweets')
# bucket_list = bucket.list()

bucket_name = 'mids-capstone-rzst'
conn = boto.connect_s3(AWS_ACCESS_KEY_ID,AWS_SECRET_ACCESS_KEY)


# bucket = conn.create_bucket(bucket_name,
#     location=boto.s3.connection.Location.DEFAULT)

testfile = "data.txt"
print 'Uploading %s to Amazon S3 bucket %s' % \
   (testfile, bucket_name)

def percent_cb(complete, total):
    sys.stdout.write('.')
    sys.stdout.flush()


# k = Key(bucket_name)
# k.key = 'data.txt'
# k.set_contents_from_filename(testfile,
#     cb=percent_cb, num_cb=10)


# s3_connection = boto.connect_s3()
bucket = conn.get_bucket('mids-capstone-rzst')
key = boto.s3.key.Key(bucket, 'data.txt')
key.set_contents_from_filename('data.txt')
key.set_acl('public-read')

Uploading data.txt to Amazon S3 bucket mids-capstone-rzst
